In [13]:
import numpy as np
import cv2
import dlib
from imutils import face_utils
from google.colab import drive
import os
from google.colab.patches import cv2_imshow

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
path = '/content/drive/MyDrive/TestOpenCV'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

['image_30.jpg',
 'image_60.jpg',
 'image_180.jpg',
 'image_270.jpg',
 'image_510.jpg',
 'models',
 'FaceImage',
 'data_face_features.pickle',
 'BaiTap32_Training.ipynb',
 'BaiTap28.ipynb']

In [19]:
# Tải mô hình đã được huấn luyện từ trước
net = cv2.dnn.readNetFromCaffe(
    path + '/models/deploy.prototxt',
    path + '/models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
)

# mo webcam
cap = cv2.VideoCapture(0)

while True:
  #doc khung hinh tu camera
  ret, frame = cap.read()

  if not ret:
    break
  # Chuẩn bị dữ liệu đầu vào
  # img: Đây là ảnh đầu vào mà bạn muốn nhận dạng khuôn mặt. Trong trường hợp của bạn, bạn đã đọc ảnh từ tệp "face.png".
  # 1.0: Đây là tỷ lệ co giãn cho ảnh. Trong trường hợp này, ảnh sẽ không bị co giãn hoặc mở rộng, và giữ nguyên kích thước ban đầu.
  # (300, 300): Đây là kích thước mà mô hình yêu cầu cho ảnh đầu vào. Mô hình mà bạn đang sử dụng mong muốn ảnh có kích thước 300x300 pixel. Do đó, bạn co giãn hoặc cắt ảnh đầu vào thành kích thước này.
  # (104, 177, 123): Đây là giá trị trung bình màu sắc được trừ đi từ mỗi pixel của ảnh. Điều này thường được sử dụng để chuẩn hóa dữ liệu đầu vào. Trong trường hợp này, các giá trị này thường được lấy từ dữ liệu huấn luyện của mô hình.
  # swapRB=False: Đây là một cờ để xác định xem có cần hoán đổi các kênh màu đỏ và xanh (Red-Blue) trong ảnh hay không. Trong trường hợp này, bạn đã đặt nó thành False, tức là không hoán đổi kênh mà
  blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 177, 123), swapRB=False)

  #dat du lieu dau vai cho mang
  net.setInput(blob)

  #chay mang de phat hien khuon mat
  faces = net.forward()

  #lay kich thuoc cua anh dau vao
  h = frame.shape[0]
  w = frame.shape[1]

  #duyet tung khuon mat da duoc phat hien
  for i in range(0, faces.shape[2]):
    confidence = faces[0, 0, i, 2]
    #kiem tra neu mat co do tin cay la 0.5
    if confidence > 0.5:
      #trich xuat toa do
      # print (faces[0,0,i, 3:7])
      startx = int(faces[0, 0, i, 3] * w)
      starty = int(faces[0, 0, i, 4] * h)
      endx = int(faces[0, 0, i, 5] * w)
      endy = int(faces[0, 0, i, 6] * h)
      #print( startx, starty, endx, endy)

      #ve hinh chu nhat xung quanh khuon mat da phat hien
      cv2.rectangle(frame, (startx, starty), (endx, endy), (0, 255, 0))

      #hien thi do tin cay
      text = 'Face: {:.2f}%'.format(confidence * 100);
      cv2.putText(frame, text, (startx, starty - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255),)

  #hien thi anh goc
  cv2.imshow('Ket qua', frame)
  if(cv2.waitKey(1) == ord('q')):
    break
cap.release()
cv2.destroyAllWindows()